In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import functools as ft
import numpy as np
import pandas as pd
from time import sleep
import re
from pprint import pprint
import time

In [2]:
today_mm, today_dd = list(map(int,time.strftime('%m.%d').split('.')))
today_mm, today_dd

(3, 16)

In [53]:
today_mm,tody_dd

(12, 31)

In [55]:
year_max, year_min = 1989,1985
year_pages = []
print('Range : %d ~ %d'%(year_min,year_max))
for info in BeautifulSoup(urlopen('https://movie.naver.com/movie/sdb/browsing/bmovie_open.nhn'),'html.parser').find('table').select('td'):
    if year_min <= int(info.text[:4]) <= year_max:
        year, pages = info.text[:4], str(np.ceil(float(info.text[7:-1])/20).astype(int))
        print('Year '+year+' - '+pages+' pages')
        year_pages.append((year,pages))
year_pages

Range : 1985 ~ 1989


[]

In [52]:
csvdata,csvtotal = [], 0
for year, pages in year_pages:
    for page in range(1,int(pages)+1):
        try:
            soup = BeautifulSoup(urlopen('https://movie.naver.com/movie/sdb/browsing/bmovie.nhn?open='+year+'&page='+str(page)),'html.parser')
            collected, total = 0, 0
            for link in soup.find('ul','directory_list').select('li > a'):
                err_msg = ''
                try:
                    date = re.findall('\d{2}.\d{2}',link.text)

                    if 'href' in link.attrs and 'class' not in link.attrs:
                        basic_page = 'https://movie.naver.com'+link.attrs['href']
                        data = BeautifulSoup(urlopen(basic_page),'html.parser')
                        total += 1

                        # title
                        title = data.find('h3','h_movie').select('a')[0].text

                        msg = 'error at date'
                        # date
                        date = data.find('dl','info_spec').find('dd').find_all('span')[-1].find_all('a')[-1].text[1:]
                        
                        msg = 'error at rating & voting'
                        # rating, vote_count
                        rating = str(ft.reduce(lambda x,y:x+y,[val.text for val in data.find('div','score score_left').select('div > a > em')[:4]]))
                        vote_count = data.find('div','score score_left').find('div','ly_count').em.text
                            
                        msg = 'error at genre'
                        # genre
                        genre = '|'.join(list(map(lambda x: x.text, filter(lambda x: 'genre' in x.attrs['href'], data.find('dl','info_spec').select('dd > p > span > a')))))
                        #genre = genre.replace('멜로/로맨스','로맨스')
                        
                        msg = 'error at plot'
                        #plot
                        plot = data.find_all('p','con_tx')[0].text
                        
                        msg = 'error at actors'
                        # actors
                        detail_page = basic_page.replace('basic','detail')
                        detail_soup = BeautifulSoup(urlopen(detail_page))

                        main_act,supp_act = '',''
                        main_num = sum([1 if line.text == '주연' else 0 for line in detail_soup.find_all('em','p_part')])
                        if main_num > 0:
                            for i,actor in enumerate(detail_soup.find('ul','lst_people').find_all('a','k_name')):
                                if i<main_num:
                                    main_act += actor.text+'|'
                                else:
                                    supp_act += actor.text+'|'
                            main_act,supp_act = main_act[:-1],supp_act[:-1]

                        msg = 'error at img'
                        #img_url
                        img_url = data.find('div','poster').find('img').attrs['src']

                        # data check => save
                        if rating != '0.00' and genre != '' and len(date) > 0:
                            '''
                            영화제목, 장르, 연도, 날짜, 평점, 투표수, 주연, 조연, url, 이미지링크
                            '''
                            csvdata.append([title, genre, year, date, rating, vote_count,
                                           plot, main_act, supp_act, basic_page,img_url])
                            collected += 1
                            csvtotal += 1
                        
                except Exception as e:
                    print('Error at title({}) & {}'.format(title,msg))
            print('year {0} page {1}/{2} - crawling {3}/{4} completed - {5}'.format(year,page,pages,collected,total,csvtotal))
            #sleep(3)
        except Exception as e:
            print('year {0} page {1}/{2} - crawling {3}/{4} failed - {5}'.format(year,page,pages,collected,total,csvtotal))
        if page % 10 == 0:
            print('Check per 10 pages - Recently appended 10 data')
            pprint(csvdata[-5:])

In [30]:
len(csvdata)

2245

In [31]:
df = pd.DataFrame(csvdata,columns=['title','genre','year','date','rating','vote_count','plot', 'main_act','supp_act','page_url','img_url'])
df.head()

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url
0,007 언리미티드,액션|스릴러,1999,12.18,8.20,163,"석유계의 거물, 로버트 킹이 폭발 사고로 죽자, 그의 딸, 일렉트라 킹(Elektr...",피어스 브로스넌,소피 마르소|로버트 칼라일|데니스 리차드|로비 콜트레인,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20111222_11/13...
1,13층,판타지|미스터리|멜로/로맨스|SF|스릴러,1999,11.27,8.58,"1,511","{“I Think, Therefore I am (나는 생각한다, 고로 존재한다)” ...",크레이그 비에코|그레첸 몰|빈센트 도노프리오|데니스 헤이스버트|스티브 슈브|아민 뮬...,제레미 로버츠|리프 휴튼|레온 리피|자넷 맥라클란,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20111223_61/13...
2,TV소설 당신,드라마,1999,04.05,6.00,1,일제 말기부터 현대에 걸쳐 살아가는 여인의 이야기,김혜선|김주승|김민희|김병세|박가령|황은하,정인선|이세창|이한나|박지영|정보석|김혜리|박상아|독고영재|김해숙,https://movie.naver.com/movie/bi/mi/basic.nave...,https://ssl.pstatic.net/static/movie/2012/06/d...
3,가든 오브 에덴,드라마,1999,04.10,7.43,14,어릴 때부터 심성이 예민한 12살의 어린 예수(킴 로시 스튜어트 분)는 목수이며 랍...,킴 로지 스튜어트|세이드 타그마오우이,보리스 테랄|마시모 기니,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20111221_78/13...
4,가방 속의 여덟 머리,코미디|범죄,1999,06.19,8.25,32,껄끄러운 일만 도맡아 처리하는 청부 살인업자 토미(Tommy Spinelli: 죠 ...,조 페시|앤디 코미우,크리스티 스완슨|토드 루이소|조지 해밀턴|다이안 캐넌|데이빗 스페이드,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20111221_38/13...


In [32]:
df.tail(5)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url
2240,혈과 사,드라마|멜로/로맨스,1990,06.09,8.86,7,스페인의 세빌랴에 사는 투우사 지망생 후안 가야르도(Juan: 크리스토퍼 라이들 분...,크리스토퍼 라이델|샤론 스톤|아나 토렌트,길레르모 몬데시노스|앨버트 비달|시몬 앤드루|안토니오 곤잘레스 플로레스|호세 루이스...,https://movie.naver.com/movie/bi/mi/basic.nave...,https://ssl.pstatic.net/static/movie/2012/06/d...
2241,호기심에 대하여,드라마,1990,07.14,7.67,3,52세의 유복한 실업가 죠지 파파 조글루는 성적 호기심이 퍽 강한 남자다. 어느 날...,,,https://movie.naver.com/movie/bi/mi/basic.nave...,https://ssl.pstatic.net/static/movie/2012/06/d...
2242,홍콩연가,멜로/로맨스|드라마,1990,01.26,4.00,1,필립은 비록 바람둥이지만 아내 메이만은 끔찍히 사랑하나. 하지만 메이는 사업핑계로 ...,주윤발|왕소봉|정문아,우양|황진선|서숙원|주보산|백문표|양해광|나란,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20111212_56/13...
2243,황금광시대,코미디|가족|모험,1990,02.17,9.04,147,금광을 찾아 알래스카에 온 찰리는 살인범 블랙의 오두막에서 금광을 찾았다는 멕케이를...,찰리 채플린|맥 스웨인|톰 머레이|헨리 버그맨,앨버트 오스틴|체스터 콘클린|베티 모리시|타이니 샌포드|밥 켈리,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20111212_135/1...
2244,흑사회,범죄|드라마,1990,06.30,6.00,10,암흑가 갱단 홍흥사의 대부 이곤이 살해된다. 이곤의 아들인 이만호는 장례식에 참석하...,주윤발|성규안,린자오 |위룡|전풍|증강|진혜민|장요양,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20210126_209/1...


In [33]:
data = df['plot'][3]
display(data)
display(re.sub('\s+', ' ', re.sub('[\r|\rxa0|\xa0]',' ', str(data).strip())).strip())

'어릴 때부터 심성이 예민한 12살의 어린 예수(킴 로시 스튜어트 분)는 목수이며 랍비인 아버지 요셉의 영향을 받으며 자란다. 18살이 될 무렵 아버지 요셉이 죽자 그는 내면의 물음에 대한 답을 구하기 위해 고향을 떠난다. 첫 도착지인 예루살렘에서 십자가에 못박혀 처형당하는 사람들을 목격하고 그 폭력의 무서움에 치를 떤다. 바로 그곳에서 알게 된 보아스와 한동안 같이 지내지만 얼마 후 보아스가 로마인들에게 역시 폭력으로 대항하는 것을 보고 모순을 느껴 예루살렘을 떠나게 된다. 인도를 비롯한 여러 곳을 여행하며 예수는 다양한 삶과 그들이 섬기는 여러 신들을 보면서 비록 모습은 다르나 본질은 동일하다는 인식을 얻는다. 신으로부터의 계시를 기다리며 낙타 모는 일을 하던 중 예수는 산적의 습격을 받은 아지즈(세드 타지마우이 분)를 구해준다. 하지만 아지즈는 예수의 은혜를 저버리고 물과 음식을 훔쳐 달아나 버린다. 모든 희망을 잃은 예수가 사막에서 쓰러졌을 때 그를 구해주는건 바로 쿰란공동체 사람들이다. 하지만 법률이나 율법보다 인간의 영혼과 존엄성을 더 존중했던 예수는 자신들의 규율 속에 갇혀있는 쿰란공동체의 나약함을 보게 된다. 이제 사신의 악한 내면인 사탄과 정면으로 대면하기 위해 사막으로 간다. 여기서 예수는 사탄을 정복하고 또한 사랑으로 감싸안는다. 그리고 진정한 하느님의 선택을 받고 사랑을 전하기 위해 세상으로 다시 돌아간다. 첫 제자 베드로와 그 가족들과 함께 길을 걷는 모습이 사랑의 말씀과 함께 나타나며 영화는 끝이 난다. 그리고 성경은 바로 여기서부터 시작된다.'

'어릴 때부터 심성이 예민한 12살의 어린 예수(킴 로시 스튜어트 분)는 목수이며 랍비인 아버지 요셉의 영향을 받으며 자란다. 18살이 될 무렵 아버지 요셉이 죽자 그는 내면의 물음에 대한 답을 구하기 위해 고향을 떠난다. 첫 도착지인 예루살렘에서 십자가에 못박혀 처형당하는 사람들을 목격하고 그 폭력의 무서움에 치를 떤다. 바로 그곳에서 알게 된 보아스와 한동안 같이 지내지만 얼마 후 보아스가 로마인들에게 역시 폭력으로 대항하는 것을 보고 모순을 느껴 예루살렘을 떠나게 된다. 인도를 비롯한 여러 곳을 여행하며 예수는 다양한 삶과 그들이 섬기는 여러 신들을 보면서 비록 모습은 다르나 본질은 동일하다는 인식을 얻는다. 신으로부터의 계시를 기다리며 낙타 모는 일을 하던 중 예수는 산적의 습격을 받은 아지즈(세드 타지마우이 분)를 구해준다. 하지만 아지즈는 예수의 은혜를 저버리고 물과 음식을 훔쳐 달아나 버린다. 모든 희망을 잃은 예수가 사막에서 쓰러졌을 때 그를 구해주는건 바로 쿰란공동체 사람들이다. 하지만 법률이나 율법보다 인간의 영혼과 존엄성을 더 존중했던 예수는 자신들의 규율 속에 갇혀있는 쿰란공동체의 나약함을 보게 된다. 이제 사신의 악한 내면인 사탄과 정면으로 대면하기 위해 사막으로 간다. 여기서 예수는 사탄을 정복하고 또한 사랑으로 감싸안는다. 그리고 진정한 하느님의 선택을 받고 사랑을 전하기 위해 세상으로 다시 돌아간다. 첫 제자 베드로와 그 가족들과 함께 길을 걷는 모습이 사랑의 말씀과 함께 나타나며 영화는 끝이 난다. 그리고 성경은 바로 여기서부터 시작된다.'

In [34]:
df['plot'] = df['plot'].map(lambda x: re.sub('\s+', ' ', re.sub('[\r|\rxa0|\xa0]',' ', str(x).strip())).strip())

In [35]:
df['vote_count'] = df['vote_count'].apply(lambda x: int(x.replace(',','')))

In [36]:
import datetime
today = datetime.datetime.today()
mmdd = str(today.month) + str(today.day)

df.to_csv('./dataset/naver_movie_dataset_90'+mmdd+'.csv',encoding='utf-8',index=False)
del df

In [37]:
df = pd.read_csv('./dataset/naver_movie_dataset_90316.csv',)
df.head(20)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url
0,007 언리미티드,액션|스릴러,1999,12.18,8.20,163,"석유계의 거물, 로버트 킹이 폭발 사고로 죽자, 그의 딸, 일렉트라 킹(Elektr...",피어스 브로스넌,소피 마르소|로버트 칼라일|데니스 리차드|로비 콜트레인,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20111222_11/13...
1,13층,판타지|미스터리|멜로/로맨스|SF|스릴러,1999,11.27,8.58,1511,"{“I Think, Therefore I m (나는 생각한다, 고로 존재한다)” -...",크레이그 비에코|그레첸 몰|빈센트 도노프리오|데니스 헤이스버트|스티브 슈브|아민 뮬...,제레미 로버츠|리프 휴튼|레온 리피|자넷 맥라클란,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20111223_61/13...
2,TV소설 당신,드라마,1999,4.05,6.00,1,일제 말기부터 현대에 걸쳐 살아가는 여인의 이야기,김혜선|김주승|김민희|김병세|박가령|황은하,정인선|이세창|이한나|박지영|정보석|김혜리|박상아|독고영재|김해숙,https://movie.naver.com/movie/bi/mi/basic.nave...,https://ssl.pstatic.net/static/movie/2012/06/d...
3,가든 오브 에덴,드라마,1999,4.10,7.43,14,어릴 때부터 심성이 예민한 12살의 어린 예수(킴 로시 스튜어트 분)는 목수이며 랍...,킴 로지 스튜어트|세이드 타그마오우이,보리스 테랄|마시모 기니,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20111221_78/13...
4,가방 속의 여덟 머리,코미디|범죄,1999,6.19,8.25,32,껄끄러운 일만 도맡아 처리하는 청부 살인업자 토미(Tommy Spinelli: 죠 ...,조 페시|앤디 코미우,크리스티 스완슨|토드 루이소|조지 해밀턴|다이안 캐넌|데이빗 스페이드,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20111221_38/13...
5,간첩 리철진,드라마|코미디,1999,5.15,7.07,190,북한의 식량난 해결을 위해 막중한 임무를 띠고 남파된 대남 공작부 요원 리철진(유오...,유오성|박인환|박진희,정영숙|신하균,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20111223_143/1...
6,개그콘서트,코미디,1999,9.11,4.78,185,1999년 9월 11일 첫 방송을 시작한 KBS2-TV의 개그 프로 사상 최장수 프...,송준근|신보라|김기리|김준현|황현희|서태훈|정경미|정지민|정은선|장기영|허경환|김기...,장동민|강성범|강승희|강유미|강일구|강주희|고혜성|곽한구|곽현화|권진영|김경아|김경...,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20121029_133/1...
7,갱 릴레이션,범죄|드라마,1999,6.26,8.17,6,대도시의 부패 경찰 디빈치(Divinci: 제임스 벨루치 분)와 로드리게즈(Rodr...,제임스 벨루시|투팍|레라 로천|데니스 퀘이드,제임스 얼 존스|데이빗 페이머|웬디 크로슨|게리 콜,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20111212_239/1...
8,건축무한육면각체의 비밀,미스터리,1999,5.01,6.54,170,"1933년에 발표된 이상의 시 ‘건축무한육면각체’, 그리고 그의 시 속에 담긴 비밀...",김태우|신은경|이민우,신성호|고구마|박정환|김재권|김명중|전해룡|김주섭,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20111221_26/13...
9,검은 고양이 흰 고양이,코미디|멜로/로맨스,1999,9.18,8.75,64,고요한 초록빛의 다뉴브 강가. 이 곳에 터를 닦아 자손을 낳고 살고 있는 두 집시 ...,스디안 토도로빅|블랑카 까틱|루비카 아조빅,자비트 메메도브,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20111223_285/1...


In [38]:
df.shape

(2245, 11)

In [39]:
c = df['rating'].mean()
m = df['vote_count'].quantile(.6)
print(round(c,3), round(m,3))

def weighted_vote_average(record):
    v, r = record['vote_count'], record['rating']
    return (v/(v+m))*r + (m/(m+v))*c

df['weighted_vote'] = df.apply(weighted_vote_average,axis=1)

7.654 49.0


### 제목 검색

In [40]:
search = '노랑머리'
df[df['title'].apply(lambda x: x.replace(' ','')).str.startswith(search.replace(' ',''))].sort_values('weighted_vote',ascending=False)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url,weighted_vote
19,노랑머리,드라마,1999,6.26,6.42,132,"머리를 노랗게 염색한 두 여자, 유나와 상희는 테크노카페에서 실직 당한 남자 영규를...",이재은|김기연|김형철,김희옥|민경진|김종훈,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20201123_264/1...,6.754072
